# Synthetic Continuous Multi-Parameter Scenario

We use this notebook to generate simulated datasets for the multi-dimensional parameter tuning solutions in the continuous action space.

In [ ]:
import copy
import itertools
import json
import pandas as pd

from IPython.display import Markdown, display
from multi_d_simulator import *

## 1. Parameters

In this section, we define how the data will look like. Some key aspects that you can control:
* Contexts
* Number of actions and the corresponding ranges
* Reward range
* Noise level
* Discretization policies
* Number of samples per context*action

In [ ]:
simulator_args = dict(
    
    # Working directory
    folder_path = r'E:\data\20200214_vector_plearning_msrnyc\data1',
    
    # Context
    contexts = {
        'platform': ['Mac', 'Windows'], 
        'network': ['wifi', 'wired'], 
        'country': ['US', 'CA']
    },
    
    # Action
    actions = {
        'x': {'mean': 2, 'min': 0, 'max': 4, 'std_range': [0.1, 2.0]},
        'y': {'mean': 1, 'min': 0, 'max': 3, 'std_range': [0.1, 2.0]}, 
        'z': {'mean': 3, 'min': 0, 'max': 5, 'std_range': [0.1, 2.0]}
    },
    discretization_fine_grain = 100,
    discretization_policy = {'x': 8, 'y': 6, 'z': 4},
    share_discretized_grid = True,
    
    # Reward
    reward_range = [0.05, 0.35],
    reward_minimization = True,
    interaction_level = 3,
    coefficient_range = [0.1, 2],
    
    # Shift from the base surface (multipliers)
    coefficient_scale_range = [0.8, 1.2], 
    dist_mean_change_range = [0.7, 1.3],
    dist_std_change_range = [0.7, 1.3],
    
    # Sample size and Noise 
    # (known_n_per_config: Int or False. If False, the sample size will estimated according to the required confidence level)
    known_n_per_config = 10,
    ci_mean = 0,
    ci_std = 0.01,
    ci_width = 0.005,
)

In [ ]:
# Generate a simulator
sim = MultiDSimulator(**simulator_args)

## 2. Generate a Base Distribution

In [ ]:
# Generate a base distribution
display(Markdown('### >>> Base Distribution'))
config_base = sim.gen_param_reward(plot=True)

# Complete Reward Formula
sim.discretize(config_base)
display(Markdown('* {0}'.format(config_base['configs']['reward_equation'])))

# Generate groundtruth data
num_values, _, _ = sim.gen_data(config_base, 1, add_error=False, plot_2d=True)

## 3. Adjust Distributions and Coefficients by Context

In [ ]:
# Initialization
config_context = {}
config_output = {}
discretized_context = {}
df_summary = pd.DataFrame()
df_all = pd.DataFrame()

In [ ]:
# Adjust
sim.random_changes()
for j, c in enumerate(sim.unique_contexts):
    
    # Context Information
    c_name = '_'.join(c)
    display(Markdown('### >>> [{0}/{1}] Generating data for context {2}'.format(j+1, len(sim.unique_contexts), c_name)))
    display(Markdown('####   Distribution'))

    # Adjust Distribution
    sim.adjust_distributuion(config_context, config_base, c, plot=True)
        
    # Adjust Coefficients
    c_coeff = sim.adjust_coefficients(c)
    
    # Ground Truth
    display(Markdown('####   Ground Truth'))
    sim.discretize(config_context[c_name], coefficients=c_coeff)
    config_context[c_name]['configs']['coefficients'] = c_coeff
    display(Markdown('* {0}'.format(config_context[c_name]['configs']['reward_equation'])))

    # Generate ground truth data
    num_values, reward_raw_min, reward_raw_max = sim.gen_data(config_context[c_name], 1, coefficients=c_coeff, add_error=False, plot_2d=True)
    
    # Generate output data by the specified discretization policies
    display(Markdown('####   Discretized Sapce'))  
    discretized_context[c_name] = copy.deepcopy(config_context[c_name])
    sim.discretize(discretized_context[c_name], discretization_policy=sim.discretization_policy, coefficients=c_coeff)
    discretized_data, _, _ = sim.gen_data(
        discretized_context[c_name], sim.n_per_config, 
        coefficients=c_coeff, add_error=True, data_min=reward_raw_min, data_max=reward_raw_max, plot_2d=True)
    config_context[c_name]['configs']['errors'] = sim.ci_dist
    
    # Save and summarize
    config_output[c_name] = sim.update_output_config(config_context[c_name])
    df_context = sim.export_data(c, discretized_data, to_file=False)
    df_all = df_all.append(df_context)
    df_summary = sim.summarize_df(df_summary, c, num_values)


## 4. Export Files

In [ ]:
df_summary.to_csv(sim.summary_file_path, index=False)
df_all.to_csv(sim.all_data_path, index=False)
json.dump(config_output, open(sim.config_path, 'w+'))